# Feature Selection
Enrique Juliá Arévalo, Sara Verde Camacho, Leo Pérez Peña

Se comienza cargando los paquetes necesarios:

In [2]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
import sys
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

**In this practical, you will become familiarized with some basic feature selection methods implemented in scikit-learn. Consider the prostate dataset that is attached to this practical. You are asked to:**

Se carga el dataset:

In [3]:
data = pd.read_csv('prostate.csv')
data.head()

,100_g_at,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,Y
0,6.927460,7.391657,3.812922,3.453385,6.070151,5.527153,5.812353,3.167275,7.354981,9.419909,...,3.770583,2.884436,2.730025,3.126168,2.870161,3.082210,2.747289,3.226588,3.480196,0
1,7.222432,7.329050,3.958028,3.407226,5.921265,5.376464,7.303408,3.108708,7.391872,10.539579,...,3.190759,2.460119,2.696578,2.675271,2.940032,3.126269,3.013745,3.517859,3.428752,1
2,6.776402,7.664007,3.783702,3.152019,5.452293,5.111794,7.207638,3.077360,7.488371,6.833428,...,3.325183,2.603014,2.469759,2.615746,2.510172,2.730814,2.613696,2.823436,3.049716,0
3,6.919134,7.469634,4.004581,3.341170,6.070925,5.296108,8.744059,3.117104,7.203028,10.400557,...,3.625057,2.765521,2.681757,3.310741,3.197177,3.414182,3.193867,3.353537,3.567482,0
4,7.113561,7.322408,4.242724,3.489324,6.141657,5.628390,6.825370,3.794904,7.403024,10.240322,...,3.698067,3.026876,2.691670,3.236030,3.003906,3.081497,2.963307,3.472050,3.598103,1


In [4]:
X = np.array(data.iloc[:, :-1]).astype(float)
y = np.array(data.iloc[:, -1]).astype(int)

1. Estimate the performance of the nearest neighbor classifier on this dataset using 10-times 10-fold cross validation when all the features are used for prediction. The number of neighbors should be chosen using an inner cross-validation procedure. You can use 5-fold cross validation for this.

El clasificador de nearest neighbor consiste en asignar una etiqueta en base a la etiqueta que tengan la mayor parte de los *k* vecinos más cercanos. Para este ejercicio será necesario, por un lado, obtener el número óptimo de vecinos. Esto se determinará haciendo una validación cruzada, con una partición en 5 de los datos. Como se han partido en 5, se repetirá el siguiente proceso 5 veces: se seleccionará uno de los grupos para test, y con los otros cuatro se ajustará el modelo para cada uno de los valores posibles de *k*, y el que dé menos error al etiquetar el de test será el que se seleccionará. Esta optimización "interna" se llevará a cabo con los datos de entrenamiento del ajuste "externo", de forma que, una vez se hayan predecido el número óptimo de vecinos, el ajuste externo de los datos de entrenamiento se hará con este número de vecinos. Este ajuste externo se repetirá 10 veces, y se utilizará la totalidad de características de los datos para ello. 

In [14]:
param_grid = {'knn__n_neighbors': np.arange(1, 21)}

pipe = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# validación cruzada externa: 10-fold
out_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# validación cruzada interna: 5-fold, para determinar el mejor numero de vecinos
in_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# validación cruzada interna: 5-fold, para determinar el mejor número de vecinos
clf = GridSearchCV(pipe, param_grid=param_grid, cv=in_cv)

# entrenar el clasificador con la validación cruzada interna
nested_scores = cross_val_score(clf, X, y, cv=out_cv)

print(f"Accuracy: {nested_scores.mean()} ± {nested_scores.std()}")


Accuracy: 0.7936363636363637 ± 0.14013275877768422


Versión más larga:

In [15]:
k_values = np.arange(3, 10) # menor rango porque tarda mucho
out_scores = []
score_per_k = {}

for out_i, (train_val_index, test_index) in enumerate(out_cv.split(X, y)):
    X_train_val, X_test = X[train_val_index], X[test_index]
    y_train_val, y_test = y[train_val_index], y[test_index]

    scaler = preprocessing.StandardScaler().fit(X_train_val)
    X_train_scaled = scaler.transform(X_train_val)
    X_test_scaled = scaler.transform(X_test) 

    mean_in_score = []

    for k in k_values:
        in_score = []

        for train_index, val_index in in_cv.split(X_train_val, y_train_val):
            X_train, X_val = X_train_val[train_index], X_train_val[val_index]
            y_train, y_val = y_train_val[train_index], y_train_val[val_index]

            scaler = preprocessing.StandardScaler().fit(X_train)
            in_X_train_scaled = scaler.transform(X_train)
            in_X_test_scaled = scaler.transform(X_val)
            
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(in_X_train_scaled, y_train)
            acc = accuracy_score(y_val, knn.predict(in_X_test_scaled))
            in_score.append(acc)
        
        mean_in_score.append(np.mean(in_score))
    
    final_k = k_values[np.argmax(mean_in_score)]
    final_knn =KNeighborsClassifier(n_neighbors= final_k)
    final_knn.fit(X_train_scaled, y_train_val)
    acc = accuracy_score(y_test, final_knn.predict(X_test_scaled))
    out_scores.append(acc)
    print(f"Iteration: {out_i + 1}, K: {final_k}, accuracy = {acc}")
    
    if final_k not in score_per_k:
        score_per_k[final_k] = [acc]
    else:
        score_per_k[final_k].append(acc)



Iteration: 1, K: 9, accuracy = 0.7272727272727273
Iteration: 2, K: 9, accuracy = 1.0
Iteration: 3, K: 9, accuracy = 0.7
Iteration: 4, K: 5, accuracy = 0.9
Iteration: 5, K: 7, accuracy = 0.8
Iteration: 6, K: 3, accuracy = 0.8
Iteration: 7, K: 7, accuracy = 1.0
Iteration: 8, K: 3, accuracy = 0.8
Iteration: 9, K: 8, accuracy = 0.8
Iteration: 10, K: 4, accuracy = 0.6


In [16]:
out_scores = np.array(out_scores)
print(f"Mean accuracy: {out_scores.mean()}")

Mean accuracy: 0.8127272727272727


In [17]:
for k in score_per_k:
    print(f"Average accuracy for {k} neighbors: {np.array(score_per_k[k]).mean()}")

Average accuracy for 9 neighbors: 0.8090909090909091
Average accuracy for 5 neighbors: 0.9
Average accuracy for 7 neighbors: 0.9
Average accuracy for 3 neighbors: 0.8
Average accuracy for 8 neighbors: 0.8
Average accuracy for 4 neighbors: 0.6


2. Estimate the performance of the nearest neighbor classifier on the same dataset when using a feature selection technique based on the F-score (ANOVA) that picks up the 10 most relevant features. Use the same cross-validation methods as in the previous step.

Se repetirá lo mismo que antes, pero en este caso, se comenzará seleccionando el número de características empleando solo las 10 más relevantes utilizando ANOVA para determinar cuáles son. ANOVA se basa en el cálculo de la varianza inter e intragrupo y las compara. Esto permite que se seleccionen las características que maximizan la varianza intergrupo, utilizando únicamente estas para el ajuste.

In [18]:
# se escalan los datos, se seleccionan las características y 
# se entrena el clasificador
pipe = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('feature_selection', SelectKBest(score_func=f_classif, k=10)),
    ('knn', KNeighborsClassifier())
])

# valores posibles para el número de vecinos
param_grid = {'knn__n_neighbors': np.arange(1, 21)}

# validación cruzada externa: 10-fold 
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# validación cruzada interna: 5-fold -> para determinar el número de vecinos
# una vez que se han seleccionado las 10 características más relevantes
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# se entrena el clasificador con la validación cruzada interna
clf = GridSearchCV(pipe, param_grid=param_grid, cv=inner_cv)

# se entrena el clasificador con la validación cruzada externa, 
# con el mejor número de vecinos (determinado por la validación cruzada interna)
nested_scores = cross_val_score(clf, X, y, cv=outer_cv)


print(f"Accuracy: {nested_scores.mean()} ± {nested_scores.std()}")


Accuracy: 0.9209090909090909 ± 0.09783186791718376


3. Repeat the previous experiment but when a random forest is used to pick up the 10 most relevant features. Use an initial filtering method based on the F-score to keep only the 20% most promising features.

Al igual que antes, se emplearán solo las 10 características más relevantes. En este caso, se determinarán a través de random forest. La utilización de random forest en la selección de características consiste en seleccionar aquellas que permiten que las hojas de los árboles sean lo más puras que sea posible. Para ello se generan árboles y se van añadiendo características. Aquellas que aumenten en mayor medida la pureza de las hojas serán las que se seleccionen. Se partirá del 20% de características, que se seleccionarán a través de un ANOVA.

In [ ]:
# validación cruzada anidada
    # se selecciona el 20% de las características con F-test
filtering = SelectKBest(score_func=f_classif, k= int(np.round(X.shape[ 1 ] * 0.2)))
    # se selecciona 10 de esas características con Random Forest

# se da a las características su importancia determinada por el Random Forest
rf_selection =  SelectFromModel(RandomForestClassifier(n_estimators = 2000, \
    random_state = 0), threshold = 0.0) 

# se define el fold externo 
n_repeats = 1
rkf = RepeatedKFold(n_splits=10, n_repeats = n_repeats, random_state=0)



In [ ]:
accuracy = np.zeros(10 * n_repeats)
np.random.seed(0)
split = 0

for train_index, test_index in rkf.split(X, y):

    print(f"Iteration #:{split + 1}")
    
    # se separan los datos en entrenamiento y test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # se escalan los datos en base a los datos de entrenamiento
    scaler.fit(X_train, y_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # se seleccionan las características con ANOVA
    filtering.fit(X_train, y_train)
    X_train_rf = filtering.transform(X_train)
    X_test_rf = filtering.transform(X_test)
    
    # se utiliza la función para ajustar los datos a Random Forest
    rf_selection.fit(X_train_rf, y_train)
    # se cambia el umbral para seleccionar las 10 características más relevantes
    rf_selection.threshold = -1.0 * np.sort(-1.0 * rf_selection.estimator_.feature_importances_)[ 9 ]
    # se seleccionan las características con Random Forest
    X_train_rf = rf_selection.transform(X_train_rf)
    X_test_rf = rf_selection.transform(X_test_rf)
    
    # se entrena el clasificador con el número de vecinos determinado por la validación cruzada interna
    knn.fit(X_train_rf, y_train)

    # se determina la precisión del clasificador
    accuracy[ split ] =  np.mean(knn.predict(X_test_rf) == y_test)
    
    split += 1

..........

In [21]:
print("Mean Accuracy KNN:%f" % np.mean(accuracy))
print("\tStd Mean Error KNN:%f" % (np.std(accuracy) / np.sqrt(len(accuracy))))

Mean Accuracy KNN:0.920000
	Std Mean Error KNN:0.027568


4. What feature selection method performs best? Can you explain why?

**Now we will address the problem of analyzing the trade-off between interpretability and prediction accuracy. For this, you are asked to:**

1. Estimate the performance of the nearest neighbor classifier with K=3 as a function of the features used for prediction. Use a 10-times 10-fold cross-validation method and plot the results obtained. That is prediction error vs. the number of features used for prediction. Use the F-score for feature selection. Report results from 1 feature to 200 features. Not all features need to be explored. Use a higher resolution when you are closer to 1 feature.

2. Repeat that process when the feature selection is done externally to the cross-validation loop using all the available data. Include these results in the previous plot.


3. Are the two estimates obtained similar? What are their differences? If they are different try to explain why this is the case.

4. By taking a look at these results, what is the optimal number of features to use in this dataset in terms of interpretability vs. prediction error?


5. Given the results obtained in this part of the practical, you are asked to indicate which particular features should be used for prediction on this dataset. Include a list with them. Take a look at the documentation of SelectKBest from scikit-learn to understand how to do this. Use all available data to provide such a list of features. 